# ADA 2018 - Homework 3



## Undestanding the StackOverflow community


Deadline: Nov 7th 2018, 23:59:59

Submission link: Check channel homework-3-public

StackOverflow is the most popular programming-related Q&A website. It serves as a platform for users to ask and answer questions and to vote questions and answers up or down. Users of StackOverflow can earn reputation points and "badges"; for example, a person is awarded 10 reputation points for receiving an "up" vote on an answer given to a question, and 5 points for the "up" vote on a question asked. Also, users receive badges for their valued contributions, which represents a kind of gamification of the traditional Q&A site. 

[Learn more about StackOverflow on Wikipedia](https://en.wikipedia.org/wiki/Stack_Overflow)

----

Dataset link:

https://drive.google.com/open?id=1POlGjqzw9v_pZ_bUnXGihOgk45kbvNjB

http://iccluster053.iccluster.epfl.ch/Posts.json.zip (mirror 1)

https://iloveadatas.com/datasets/Posts.json.zip (mirror 2)

Dataset description:

* **Id**: Id of the post
* **CreationDate**: Creation date of the post (String format)
* **PostTypeId**: Type of post (Question = 1, Answer = 2)
* **ParentId**: The id of the question. Only present if PostTypeId = 2
* **Score**: Points assigned by the users
* **Tags**: Tags of the question. Only present if PostTypeId = 1
* **Title**: Only present if PostTypeId = 1
* **ViewCount**: Only present if PostTypeId = 1

The dataset format is JSON. Here are examples of a question and an answer:

Question:
```json
{
    "Id": 10130734,
    "CreationDate": "2012-04-12T19:51:25.793+02:00",
    "PostTypeId": 1,
    "Score": 4,
    "Tags": "<python><pandas>",
    "Title": "Best way to insert a new value",
    "ViewCount": 3803
}
```

Answer:
```json
{  
   "CreationDate":"2010-10-26T03:19:05.063+02:00",
   "Id":4020440,
   "ParentId":4020214,
   "PostTypeId":2,
   "Score":1
}
```

----
Useful resources:

**Spark SQL, DataFrames and Datasets Guide**

https://spark.apache.org/docs/latest/sql-programming-guide.html

**Database schema documentation for the public data dump**

https://meta.stackexchange.com/questions/2677/database-schema-documentation-for-the-public-data-dump-and-sede

----

**Note:** Use Spark where possible. Some computations can take more than 10 minutes on a common notebook. Consider to save partial results on disk.

In [2]:
# Add your imports here
import pandas as pd
import numpy as np
import scipy as sp

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark import SparkContext

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import requests

spark = SparkSession.builder.getOrCreate()

DATA_FOLDER = 'data/'

### Task A: Convert the dataset to a more convenient format
As a warm-up task (and to avoid to warm up your laptop too much), load the dataset into a Spark dataframe, show the content, and save it in the _Parquet_ format. Use this step to convert the fields to a more convenient form.

Answer the following questions:

1. How many questions have been asked on StackOverflow?
2. How many answers have been given?
3. What is the percentage of questions with a score of 0?

**Hint:** The next tasks involve a time difference. Consider storing time in numeric format.

In [7]:
#Upload in a more beautiful way
# data_stack = spark.read.json(requests.get("https://iloveadatas.com/datasets/Posts.json.zip"))
#May not work beacause of the .zip -> TODO : find how or juste assume that we load into our folder the data set

#Upload data if they are in the folder ./data/
data_stack = spark.read.json(DATA_FOLDER + 'Posts.json')
data_stack.write.parquet("data_stack.parquet")
data_stack = spark.read.parquet("data_stack.parquet")
data_stack_rdd = data_stack.rdd

In [8]:
data_stack.show()

+--------------------+--------+--------+----------+-----+--------------------+--------------------+---------+
|        CreationDate|      Id|ParentId|PostTypeId|Score|                Tags|               Title|ViewCount|
+--------------------+--------+--------+----------+-----+--------------------+--------------------+---------+
|2017-08-17T16:20:...|45740344|45740224|         2|    0|                null|                null|     null|
|2017-08-17T16:20:...|45740346|45739185|         2|    1|                null|                null|     null|
|2017-08-17T16:20:...|45740348|    null|         1|    2|<flash><react-nat...|Is it possible to...|      143|
|2017-08-17T16:20:...|45740350|45739102|         2|    1|                null|                null|     null|
|2017-08-17T16:20:...|45740352|42473616|         2|    0|                null|                null|     null|
|2017-08-17T16:20:...|45740354|45668191|         2|    1|                null|                null|     null|
|2017-08-1

In [20]:
number__of_question = data_stack_rdd.filter(lambda x : x.PostTypeId == 1).count()
print("The number of question in StackOverflow : {}".format(number__of_question))

The number of question in StackOverflow : 15647060


In [24]:
number__of_answer = data_stack_rdd.filter(lambda x : x.PostTypeId == 2).count()
print("The number of answer in StackOverflow : {}".format(number__of_answer))

The number of answer in StackOverflow : 25192772


In [35]:
number_of_question_score_0 = data_stack_rdd.filter(lambda x : (x.PostTypeId == 1) and (x.Score == 0)).count()
print("The proportion of question with a null score in StackOverflow : {}"\
      .format(str(number_of_question_score_0/number__of_question)[:6]))

The proportion of question with a null score in StackOverflow : 0.4654


In [29]:
#for Task B
data_stack.select("Tags").write.save("Tags.parquet")

In [31]:
#for Task C
data_stack.select("Score", "ViewCount", "Tags").write.save("Score_View_Tags.parquet")

**Hint:** Load the dataset from the Parquet file for the next tasks.

### Task B: What are the 10 most popular tags?

What are the most popular tags in StackOverflow? Use Spark to extract the information you need, and answer the following questions with Pandas and Matplotlib (or Seaborn):

1. What is the proportion of tags that appear in fewer than 100 questions?
2. Plot the distribution of the tag counts using an appropriate representation.
3. Plot a bar chart with the number of questions for the 10 most popular tags.

For each task describe your findings briefly.

In [37]:
#We have to sample?
df_sampled = data_stack.sample(False, 0.0001, 42)
rdd_posts = df_sampled.rdd
rdd_posts.count()

4054

In [39]:
rdd_tags = data_stack_rdd.filter(lambda r: r[5] != None).map(lambda r: (r[5][1:-1].split('><')))

#if with sample
#rdd_tags = rdd_posts.filter(lambda r: r[5] != None).map(lambda r: (r[5][1:-1].split('><')))

In [40]:
rdd_tags2 = rdd_tags.countByKey()
df_tags = pd.DataFrame.from_dict(rdd_tags2, orient='index')
df_tags = df_tags.rename(index=str, columns={0: "count"})
df_tags = df_tags.sort_values('count',ascending=False)
df_tags.head(5)

,count
javascript,1535788
java,1340350
c#,1158041
php,1049934
python,929698


In [44]:
total_tags = len(df_tags)
df_tags_less_100 = df_tags[df_tags['count'] < 100]
total_tags_less_100 = len(df_tags_less_100)
print('Proportion of tags appearing in fewer than 100 questions : {} %'.format(str(total_tags_less_100*100/total_tags)[:5]))

Proportion of tags appearing in fewer than 100 questions : 90.05 %


### Task C: View-score relation

We want to investigate the correlation between the view count and the score of questions.

1. Get the view count and score of the questions with tag ```random-effects``` and visualize the relation between these two variables using an appropriate plot.
2. Are these two variables correlated? Use the Pearson coefficient to validate your hypothesis. Discuss your findings in detail.

**Hint:** Inspect the data visually before drawing your conclusions.

In [ ]:
# Add your code and description here

### Task D: What are the tags with the fastest first answer?

What are the tags that have the fastest response time from the community? We define the response time as the difference in seconds between the timestamps of the question and of the first answer received.

1. Get the response time for the first answer of the questions with the tags ```python``` and ```java```.
2. Plot the two distributions in an appropriate format. What do you observe? Describe your findings and discuss the following distribution properties: mean, median, standard deviation.
3. We believe that the response time is lower for questions related to Python (compare to Java). Contradict or confirm this assumption by estimating the proper statistic with bootstrapping. Visualize the 95% confidence intervals with box plots and describe your findings.
3. Repeat the first analysis (D1) by using the proper statistic to measure the response time for the tags that appear at least 5000 times. Plot the distribution of the 10 tags with the fastest response time.


In [ ]:
# Add your code and description here

### Task E: What's up with PySpark?
The number of questions asked regarding a specific topic reflect the public’s interest on it. We are interested on the popularity of PySpark. Compute and plot the number of questions with the ```pyspark``` tag for 30-day time intervals. Do you notice any trend over time? Is there any correlation between time and number of questions?


In [ ]:
# Add your code and description here